# Projet maison n° 4

In [12]:
# imports
import requests
from bs4 import BeautifulSoup
import re

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [13]:
r = requests.get(URL_PAGE2)

In [14]:
soup = BeautifulSoup(r.content)

In [236]:
'''r = requests.get(URL_PAGE3)
soup = BeautifulSoup(r.content)

database = []
for subcontent in soup.findAll('ul', attrs={'class': 'pricing-table-list'}):
    for subcontent in tag.findAll('li'):
        y = re.search(r'([0-9]+) database', subcontent.text.strip())
        if y: 
            database.append(y.group(1))
    print(database)'''

"r = requests.get(URL_PAGE3)\nsoup = BeautifulSoup(r.content)\n\ndatabase = []\nfor subcontent in soup.findAll('ul', attrs={'class': 'pricing-table-list'}):\n    for subcontent in tag.findAll('li'):\n        y = re.search(r'([0-9]+) database', subcontent.text.strip())\n        if y: \n            database.append(y.group(1))\n    print(database)"

In [245]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    price = []
    for content in soup.findAll('div', attrs={'class': 'pricing-table-header'}):
        rx = re.compile('<span class="pricing-table-price">([^<]+)<span>')
        for match in rx.finditer(str(content)):
            price.append(match.group(1).strip())
            
    storage = []
    for subcontent in soup.findAll('ul', attrs={'class': 'pricing-table-list'}):
        storage.append(subcontent.select('li')[3].text[:4].strip()) 
        
    database = []
    for tag in soup.findAll('div', attrs={'class': 'pricing-tables pure-g'}):
        for li in tag.findAll('li'):
            y = re.search(r'([0-9]+) database', li.text.strip())
            if y:
                database.append(int(y.group(1)))
        
    categories = []
    for content in soup.findAll('div', attrs={'class': 'pricing-table-header'}):
        category = content.find('h2').text
        categories.append(category)
        
    prices = {}
    for i in range(0, len(categories)):
        prices[categories[i]] = {
        'price': price[i],
        'storage': storage[i],
        'databases': database[i]}  
    
    return prices

In [246]:
get_prices_from_url("https://kim.fspot.org/cours/page2.html")

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100G', 'databases': 25}}

In [247]:
get_prices_from_url("https://kim.fspot.org/cours/page3.html")

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100G', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [277]:
r = requests.get('https://www.beerwulf.com/fr-fr/p/bieres/affligem-blanche-fut-5L')
soup = BeautifulSoup(r.content)
for beer_info in soup.findAll('div', attrs={'class': 'product-info beer-product-info'}):
    degree = 0.0
    for beer_degree in beer_info.findAll('dd'):
        beer_degree = re.search(r'([0-9]+,[0-9]+)%', beer_degree.text.strip())
        if beer_degree:
            degree = float(beer_degree.group(1).replace(',', '.'))
            print(degree)

4.8


In [286]:
# partie B-1
def extract_beer_infos(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    name = ""
    for beer_name in soup.findAll('div', attrs={'class': 'product-detail-info-title'}):
        name = beer_name.find('h1').text
        
    for beer_info in soup.findAll('div', attrs={'class': 'product-info beer-product-info'}):
        style = ""
        for beer_style in beer_info.findAll('a'):
            style = beer_style.text.strip()
            
        volume = 0
        for beer_volume in beer_info.findAll('dd', attrs={'class': 'small-6 medium-9 columns js-beer-volume'}):
            volume = int(beer_volume.text.strip()[0:2])
            
        degree = 0.0
        for beer_degree in beer_info.findAll('dd'):
            beer_degree = re.search(r'([0-9]+,[0-9]+)%', beer_degree.text.strip())
            if beer_degree:
                degree = float(beer_degree.group(1).replace(',', '.'))
                
        origin = ""
        for beer_origin in beer_info.findAll('dd', attrs={'class': 'small-6 medium-9 columns js-beer-country'}):
            origin = beer_origin.text.strip()
        
        brasseur = ""
        brasseur = beer_info.findAll('dd')[4].text.strip()
    
    
    beer_infos = {
        'Nom': name,
        'Style': style,
        'Contenu': volume,
        'Degré d’alcool': degree,
        'Origine': origin,
        'Brasseur': brasseur
    }
    return beer_infos

In [287]:
# partie B-2
#URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France"

def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    
    r = requests.get(URL_BEERLIST_FRANCE)
    content = r.json()
    
    for item in content['items']:
        beer_pages.append(item['contentReference'])
        
    # Sequential version (slow):
    beers = []
    
    for page in beer_pages:
        beers.append(extract_beer_infos('https://www.beerwulf.com/' + page))

    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

In [288]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [289]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.195s

OK
